In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"

s3_bucket = "s3://mist-secorapp-{env}/rrm-local/rrm-local-{env}/".format(env=env)

date_day = "2020-03-2*"
hr = '*'

rrm_local_path = "dt={day}/hr={hr}/*.seq".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)




s3://mist-secorapp-production/rrm-local/rrm-local-production/dt=2020-03-2*/hr=*/*.seq


In [2]:
# rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)
rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)
df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
df_rrm.printSchema()

/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


root
 |-- ap_id: string (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- channel_update_interval: long (nullable = true)
 |-- clients: long (nullable = true)
 |-- command: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- power: long (nullable = true)
 |-- pre_bandwidth: long (nullable = true)
 |-- pre_channel: long (nullable = true)
 |-- pre_power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- rrm_bandwidth: long (nullable = true)
 |-- rrm_channel: long (nullable = true)
 |-- rrm_enabled: boolean (nullable = true)
 |-- rrm_power: long (nullable = true)
 |-- site_id: string (nullable = true)
 |-- source_data: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- updated: boolean (nullable = true)
 |-- when: string (nullable = true)



In [3]:
rdd_rrm_local.take(2)

[(61532737,
  bytearray(b'{"updated": true, "clients": 1, "pre_bandwidth": 20, "power": 12, "timestamp": 1584662401, "site_id": "87898131-7246-498e-896e-b89d170902dc", "org_id": "20101562-d4f8-41aa-aaa9-98349d78dfa0", "band": "24", "bandwidth": 20, "rad_id": "r24", "rrm_bandwidth": 20, "rrm_power": 12, "rrm_channel": 11, "pre_power": 12, "when": "2020-03-20T00:00:01", "rrm_enabled": true, "reason": "non-wifi", "ap_id": "5c5b35ae81dd", "source_data": "redis-neighbor-hourly", "command": "capacity-anomaly", "channel_update_interval": 7200, "pre_channel": 1, "channel": 11}')),
 (61532738,
  bytearray(b'{"updated": true, "clients": 1, "pre_bandwidth": 20, "power": 8, "timestamp": 1584662401, "site_id": "8d06fe54-07e7-462b-ab0d-4432ba86dc42", "org_id": "20101562-d4f8-41aa-aaa9-98349d78dfa0", "band": "24", "bandwidth": 20, "rad_id": "r24", "rrm_bandwidth": 20, "rrm_power": 8, "rrm_channel": 11, "pre_power": 8, "when": "2020-03-20T00:00:01", "rrm_enabled": true, "reason": "non-wifi", "ap_id": 

In [4]:
df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
df_rrm.show()

+------------+----+---------+-------+-----------------------+-------+----------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|       ap_id|band|bandwidth|channel|channel_update_interval|clients|         command|              org_id|power|pre_bandwidth|pre_channel|pre_power|rad_id|        reason|rrm_bandwidth|rrm_channel|rrm_enabled|rrm_power|             site_id|         source_data| timestamp|updated|               when|
+------------+----+---------+-------+-----------------------+-------+----------------+--------------------+-----+-------------+-----------+---------+------+--------------+-------------+-----------+-----------+---------+--------------------+--------------------+----------+-------+-------------------+
|5c5b35ae81dd|  24|       20|     11|                   7200|      1|capacity-anomaly|20101562-d4

In [5]:
df_rrm.printSchema()

root
 |-- ap_id: string (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- channel_update_interval: long (nullable = true)
 |-- clients: long (nullable = true)
 |-- command: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- power: long (nullable = true)
 |-- pre_bandwidth: long (nullable = true)
 |-- pre_channel: long (nullable = true)
 |-- pre_power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- rrm_bandwidth: long (nullable = true)
 |-- rrm_channel: long (nullable = true)
 |-- rrm_enabled: boolean (nullable = true)
 |-- rrm_power: long (nullable = true)
 |-- site_id: string (nullable = true)
 |-- source_data: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- updated: boolean (nullable = true)
 |-- when: string (nullable = true)



In [6]:
df_USNYELE = df_rrm.filter((col('site_id')==user_site_USNYELE) & (col('band')==5))
df_USFLLON = df_rrm.filter((col('site_id')==user_site_USFLLON) & (col('band')==5))

NameError: name 'user_site_USNYELE' is not defined

In [ ]:
df_USNYELE.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.filter(col("command")=="rrm-global").groupBy("band",'when', "command", 'reason').count().show()